In [1]:
from qdots_qll.all_funcs import *
from qdots_qll.process_representations import *

In [4]:
omega_ket = [qu.tensor(qu.basis(2, i), qu.basis(2, i))/np.sqrt(2) for i in range(2)]

In [22]:
ijmat = np.zeros([2, 2, 2, 2], dtype=np.complex64)

In [23]:
qu.basis(2, 0)*qu.basis(2, 0).dag()

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[1. 0.]
 [0. 0.]]

In [24]:
for i in range(2):
    for j in range(2):
        ijmat[i, j] = (qu.basis(2, i)*qu.basis(2, j).dag()).full()

In [27]:
ijmat[0, 1]

array([[0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j]], dtype=complex64)

In [28]:
vec(ijmat[0, 1])

Array([[0.+0.j],
       [0.+0.j],
       [1.+0.j],
       [0.+0.j]], dtype=complex64)

In [30]:
import joblib

true_pars = joblib.load("true_vals.job")

In [32]:
N = 2
G = np.array(
    [qu.identity(N), qu.sigmax(), qu.sigmay(), qu.sigmaz()]
) / np.sqrt(N)



# Basic model

delta = 0.12739334807998307
Omega = 0.5

A = np.array([[1, 0], [0, 0]])
sigmap = np.array([[0, 1], [0, 0]])
sigman = np.array([[0, 0], [1, 0]])

H0 = delta * A + Omega / 2 * (sigmap + sigman)

measurement_ops = vec(np.array([[1, 0], [0, 0]]))

true_vals = joblib.load("true_vals.job")

rho0 = vec(np.array([[1, 0], [0, 0]]))

bnds = jnp.array([[-0.2, 0.2],
                  [-1, 1],
                  [-0.1, 0.1],
                  [-0.2, 0.2],
                  [-0.2, 0.2],
                  [-0.2, 0.2],
                  [-0.2, 0.2],
                  [-0.2, 0.2]
                  ])

In [36]:
import h5py


def import_hdf5(filename):
    f = h5py.File(filename, "r")
    a_group_key = list(f.keys())[0]
    data = list(f[a_group_key])
    return np.array(data)


arr = import_hdf5("numerical_L_true_mathematica.hdf5")

true_L = arr["Re"] + 1j * arr["Im"]

In [37]:
mat_gg = make_diss_gjgi(G, N)

In [38]:
phys_model = {
    "Dim of H space": d,
    "H": H0,
    "True Lindbladian": true_L,
    "Measurement operators": measurement_ops,
    "Initial state": rho0,
    "Identity matrix vectorized": vec(jnp.eye(d)),
    "Identity matrix": jnp.eye(d),
    "Mat_gg": mat_gg,
}

# model = {
#     "Number of parameters": 8,
#     "Space boundaries": bnds,  #jnp.array([[-1, 1] for _ in range(8)]),
#     "Number of particles": 500,
#     "Number of experimental repetitions": 5,
#     "physical_model": phys_model,
# }

In [121]:
true_L

array([[ 0.        +0.00000000e+00j,  0.        +2.50000000e-01j,
         0.        -2.50000000e-01j,  0.        +0.00000000e+00j],
       [-0.00411732+2.27598191e-01j, -0.01134756-2.71050543e-20j,
         0.        +0.00000000e+00j, -0.00122611-2.30595655e-01j],
       [-0.00411732-2.27598191e-01j,  0.        +0.00000000e+00j,
        -0.01134756+2.71050543e-20j, -0.00122611+2.30595655e-01j],
       [ 0.        +0.00000000e+00j,  0.        -2.50000000e-01j,
         0.        +2.50000000e-01j,  0.        +0.00000000e+00j]])

In [42]:
generate_liouvillian_particle(true_pars, phys_model).round(3)

Array([[ 0.   +0.j        ,  0.   +0.25j      ,  0.   -0.25j      ,
         0.   +0.j        ],
       [-0.004+0.22800002j, -0.011+0.j        ,  0.   +0.j        ,
        -0.001-0.231j     ],
       [-0.004-0.22800002j,  0.   +0.j        , -0.011-0.j        ,
        -0.001+0.231j     ],
       [ 0.   +0.j        , -0.   -0.25j      , -0.   +0.25j      ,
         0.   +0.j        ]], dtype=complex64)

In [45]:
jax.grad(generate_liouvillian_particle, 0)(true_pars, phys_model)

TypeError: Gradient only defined for scalar-output functions. Output had shape: (4, 4).

In [48]:
beta00 = (qu.tensor(qu.basis(2, 0), qu.basis(2, 0)) + qu.tensor(qu.basis(2, 1), qu.basis(2, 1)))/np.sqrt(2)
beta10 = (qu.tensor(qu.basis(2, 0), qu.basis(2, 0)) - qu.tensor(qu.basis(2, 1), qu.basis(2, 1)))/np.sqrt(2)

beta01 =(qu.tensor(qu.basis(2, 0), qu.basis(2, 1)) + qu.tensor(qu.basis(2, 1), qu.basis(2, 0)))/np.sqrt(2)
beta11 = (qu.tensor(qu.basis(2, 0), qu.basis(2, 1)) - qu.tensor(qu.basis(2, 1), qu.basis(2, 0)))/np.sqrt(2)


In [51]:
list_beta = [beta00, beta10, beta01, beta11]

In [54]:
np.sum(, axis=0)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [56]:
povm_bell = np.array([i*i.dag() for i in list_beta])

In [58]:
povm_bell[0]

array([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]])

In [ ]:
particle, E_0, t, model)

In [59]:
p_E0_particle(true_pars, vec(povm_bell[0]), 0.5, phys_model)

TypeError: dot_general requires contracting dimensions to have the same shape, got (4,) and (16,).

In [134]:
old = np.array([[ 9.9489367e-01+5.1879169e-12j, -1.2316820e-03-6.7929231e-02j,
         1.0014700e+07+4.0849461e+01j, -4.7787153e+03+1.3307250e+08j],
       [-1.2316821e-03+6.7929223e-02j,  5.1063532e-03-5.1879169e-12j,
        -4.3516362e+03+6.8909325e+05j, -1.0014699e+07-4.0849464e+01j],
       [-1.0014698e+07-4.5107430e+01j,  4.3457217e+03+6.8909325e+05j,
         5.1736026e-03+1.6769558e-12j, -3.6864163e-04+6.8823859e-02j],
       [ 3.6365447e+03+1.3307249e+08j,  1.0014699e+07+4.5107430e+01j,
        -3.6864163e-04-6.8823852e-02j,  9.9482638e-01-1.6769558e-12j]])

In [152]:
@jit
def _unvec(vec_rho):
    return vec_rho.reshape((2, 2), order="F")


@jit
def evolve_rho0_particle(particle, t, phys_model, rho0):
    L = generate_liouvillian_particle(particle, phys_model)
    return _unvec(v_evolve(t, L, rho0))


def get_choi(particle, t, phys_model, ijmat):
    # ij_evolved = np.zeros([2,2, 2, 2], dtype=np.complex64)

    suma = []
    for i in range(2):
        aux_list = []
        for j in range(2):
            # ij_evolved[i, j] = evolve_rho0_particle(particle, t ,phys_model, vec(ijmat[i, j]))
            aux_list.append(
                evolve_rho0_particle(particle, t, phys_model, vec(ijmat[i, j]))
            )
        suma.append(aux_list)

    choi = 0
    for i in range(2):
        for j in range(2):
            # choi = choi + np.kron(ijmat[i, j], ij_evolved[i, j])
            choi = choi + jnp.kron(ijmat[i, j], suma[i][j])
    return choi


def get_p_Ei(particle, t, phys_model, ijmat, E_i):
    rho = get_choi(particle, t, phys_model, ijmat)
    return jnp.real(jnp.trace(E_i @ rho))

In [145]:
get_choi(true_pars, 0.3, phys_model, ijmat)

Array([[ 9.9489367e-01+5.1879169e-12j, -1.2316820e-03-6.7929231e-02j,
         1.0014700e+07+4.0849461e+01j, -4.7787153e+03+1.3307250e+08j],
       [-1.2316821e-03+6.7929223e-02j,  5.1063532e-03-5.1879169e-12j,
        -4.3516362e+03+6.8909325e+05j, -1.0014699e+07-4.0849464e+01j],
       [-1.0014698e+07-4.5107430e+01j,  4.3457217e+03+6.8909325e+05j,
         5.1736026e-03+1.6769558e-12j, -3.6864163e-04+6.8823859e-02j],
       [ 3.6365447e+03+1.3307249e+08j,  1.0014699e+07+4.5107430e+01j,
        -3.6864163e-04-6.8823852e-02j,  9.9482638e-01-1.6769558e-12j]],      dtype=complex64)

In [146]:
np.sum(povm_bell, 0)

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [147]:
povm_bell[0]

array([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
       [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]])

In [148]:
L = np.array(generate_liouvillian_particle(true_pars, phys_model))

In [149]:
L[np.abs(L)< 0.000001] = 0

In [150]:
np.linalg.eig(to_choi_from_super(true_L))[0]

array([-0.48482055+1.34738567e-17j,  0.47347341-9.30541103e-18j,
        0.02749808+1.88053151e-17j, -0.01615094-1.43001435e-17j])

In [156]:
jax.grad(get_p_Ei, 0)(true_pars, 0.3, phys_model, ijmat, povm_bell[3] )

Array([-1.3107200e+05,  8.2239570e+03,  8.1920000e+03,  1.2309384e+02,
        2.8813200e+00,  4.0705916e+03,  2.6562500e-01, -5.2641992e+02],      dtype=float32)

In [162]:
jax.grad(get_p_Ei, 0)(true_pars, 11, phys_model, ijmat, povm_bell[2] )

Array([ 7.3400320e+06,  8.8910904e+00,  2.5165824e+07,  4.2389850e+06,
       -9.0112164e+04,  4.6175211e+04, -6.5000000e+00, -5.3354762e+05],      dtype=float32)

In [66]:
ex = jnp.array([[ 0.99489367+5.187917e-12j],
       [-0.00123168+6.792922e-02j],
       [-0.00123168-6.792923e-02j],
       [ 0.00510635-5.187917e-12j]])

Array([[ 0.99489367+5.187917e-12j, -0.00123168-6.792923e-02j],
       [-0.00123168+6.792922e-02j,  0.00510635-5.187917e-12j]],      dtype=complex64)